In [1]:
import tqdm
import fs.zipfs
import fs.multifs
import os
import fs.copy
import sys
sys.path.append("/scr/BEHAVIOR-1K/asset_pipeline")
import b1k_pipeline.utils

OUT_FILENAME = "/scr/rc17.zip"

# In the order of priority
PARALLELS = [
    "/scr/rc16.zip",
    "/scr/rc17_patch.zip",
]

OBJECTS_TO_REMOVE = []

def main():
    # Get a multi-FS view over all of the parallel filesystems.
    multi_fs = fs.multifs.MultiFS()
    for priority, parallel_zip_name in enumerate(PARALLELS):
        print("Adding", parallel_zip_name)
        multi_fs.add_fs(os.path.basename(parallel_zip_name), fs.zipfs.ZipFS(parallel_zip_name), priority=priority)

    # Copy all the files to the output zip filesystem.
    print("Copying files")
    total_files = sum(1 for f in multi_fs.walk.files())
    with b1k_pipeline.utils.WriteOnly7ZipFS(OUT_FILENAME) as out_fs:
        with tqdm.tqdm(total=total_files) as pbar:
            fs.copy.copy_fs(multi_fs, out_fs, on_copy=lambda *args: pbar.update(1))

        print("Removing some objects")
        objects_dir = out_fs.opendir("objects")
        for obj_to_remove in OBJECTS_TO_REMOVE:
            objdir_glob = [x.path for x in objects_dir.glob(f"*/{obj_to_remove}/")]
            assert len(objdir_glob) == 1, f"Needed exactly one dir for {obj_to_remove}, got {objdir_glob}"
            objdir = objdir_glob[0]
            print("Removing", objdir)
            objects_dir.removetree(objdir)

        systems_dir = out_fs.opendir("systems")
        for system_dir in list(systems_dir.listdir("/")):
            if objects_dir.exists(system_dir):
                print("Removing", system_dir)
                objects_dir.removetree(system_dir)

main()

Adding /scr/rc16.zip
Adding /scr/rc17_patch.zip
Copying files


100%|██████████| 210126/210126 [03:07<00:00, 1118.41it/s]


Removing some objects


In [2]:
! gsutil cp "/scr/rc17.zip" "gs://gibson_scenes/og_dataset_1_2_0rc17.zip"

Copying file:///scr/rc17.zip [Content-Type=application/zip]...
==> NOTE: You are uploading one or more large file(s), which would run          
significantly faster if you enable parallel composite uploads. This
feature can be enabled by editing the
"parallel_composite_upload_threshold" value in your .boto
configuration file. However, note that if you do this large files will
be uploaded as `composite objects
<https://cloud.google.com/storage/docs/composite-objects>`_,which
means that any user who downloads such objects will need to have a
compiled crcmod installed (see "gsutil help crcmod"). This is because
without a compiled crcmod, computing checksums on composite objects is
so slow that gsutil disables downloads of composite objects.

| [1 files][ 29.5 GiB/ 29.5 GiB]  103.4 MiB/s                                   
Operation completed over 1 objects/29.5 GiB.                                     
